In [ ]:
from trainfuncs import *

## Train Base From GPT2 With All Data

In [ ]:
model, token = new_model("gpt2")
dataset = make_dataset(get_workouts() + get_posts(), token)

In [ ]:
stats = train_model(
    model,
    dataset,
    epochs=5,
    batch_size=8,
    learning_rate=5e-4,
    sample_every=5000,
)

In [ ]:
plot_df_stats(stats)

In [ ]:
save_model(
    model,
    token,
    "gitfit-model-base",
)

## Train With Workouts From Base

In [ ]:
model, token = new_model(
    "cluffa/gitfit-model-base",
    # "gitfit-model-base/"
)
dataset = make_dataset(get_workouts(), token)

In [ ]:
stats = train_model(
    model,
    dataset,
    epochs=15,
    batch_size=8,
    learning_rate=5e-6,
    sample_every=200,
)

In [ ]:
plot_df_stats(stats)

In [ ]:
save_model(
    model,
    token,
    "gitfit-model",
)